In [1]:
import numpy as np

In [59]:
with open('hw8_ratings_fa18.txt') as f:
    rating = f.readlines()    

with open('hw8_movieTitles_fa18.txt') as f:
    movies = f.readlines()
for index,item in enumerate(movies):
    movies[index] = item.strip()

with open('hw8_studentPIDs_fa18.txt') as f:
    students = f.readlines()
for index,item in enumerate(students):
    students[index] = item.strip()
ratingM = np.zeros((269,62))
for index,item in enumerate(rating):
    temp = item.strip().split(' ')
    for j,value in enumerate(temp):
        if value == '1':
            ratingM[index,j] = 1
        elif value =='0':
            ratingM[index,j] = 0
        elif value =='?':
            ratingM[index,j] = -1

In [56]:
popular = []
for i in range(len(movies)):
    popular.append(np.sum((ratingM[:,i] == 1).astype(np.int64))/(269-np.sum((ratingM[:,i] == -1).astype(np.int64))))
index = range(0,62)     
combine = list(zip(index,popular))
combine = sorted(combine,key=lambda x:x[1])
for i in combine:
    print(movies[i[0]])

The_Last_Airbender
Fifty_Shades_of_Grey
I_Feel_Pretty
Chappaquidick
Man_of_Steel
Prometheus
The_Shape_of_Water
Phantom_Thread
Magic_Mike
World_War_Z
Bridemaids
American_Hustle
Drive
The_Hunger_Games
Thor
Pitch_Perfect
Fast_Five
Avengers:_Age_of_Ultron
Jurassic_World
The_Hateful_Eight
The_Revenant
Dunkirk
Star_Wars:_The_Force_Awakens
Mad_Max:_Fury_Road
Captain_America:_The_First_Avenger
The_Perks_of_Being_a_Wallflower
Iron_Man_2
La_La_Land
Manchester_by_the_Sea
The_Help
Midnight_in_Paris
The_Girls_with_the_Dragon_Tattoo
21_Jump_Street
Frozen
Now_You_See_Me
X-Men:_First_Class
Ex_Machina
Harry_Potter_and_the_Deathly_Hallows:_Part_1
Toy_Story_3
Her
The_Great_Gatsby
The_Avengers
The_Theory_of_Everything
Room
Gone_Girl
Three_Billboards_Outside_Ebbing
Les_Miserables
Harry_Potter_and_the_Deathly_Hallows:_Part_2
The_Martian
Avengers:_Infinity_War
Darkest_Hour
Hidden_Figures
12_Years_a_Slave
Ready_Player_One
Black_Swan
Django_Unchained
Wolf_of_Wall_Street
Shutter_Island
Interstellar
The_Dark_Kni

In [111]:
with open('hw8_probZ_init.txt') as f:
    probZ = f.readlines()
for index,item in enumerate(probZ):
    probZ[index] = float(item.strip())
probZ = np.array(probZ)

with open('hw8_probRgivenZ_init.txt') as f:
    probRZ = f.readlines()
probRZM = []
for index,item in enumerate(probRZ[:-1]):
    temp = item.strip().split('   ')
    probRZM.append(list(map(lambda x:float(x),temp)))
probRZM = np.array(probRZM)

In [112]:
def post(zvalue,student,probZ,probRZM,ratingM):
    nume = probZ[zvalue]
    for j in range(62):
        if(ratingM[student,j]==1):
            nume *= probRZM[j,zvalue]
        elif(ratingM[student,j]==0):
            nume *= (1- probRZM[j,zvalue])
    demo = 0
    for k in range(4):
        temp = probZ[k]
        for j in range(62):
            if(ratingM[student,j]==1):
                temp *= probRZM[j,k]
            elif(ratingM[student,j]==0):
                temp *= (1- probRZM[j,k])
        demo += temp
    return nume/demo

def LFun(probZ,probRZM,ratingM):
    res = 0
    for t in range(269):
        mysum = 0
        for i in range(4):
            temp = probZ[i]
            for j in range(62):
                if(ratingM[t,j]==1):
                    temp *= probRZM[j,i]
                elif(ratingM[t,j]==0):
                    temp *= (1- probRZM[j,i]) 
            mysum += temp
        res += np.log(mysum)
    return res/269
        
        
check = [0,1,2,4,8,16,32,64,128]    
for time in range(129):
    if time in check:
        print("Iterations:{},log-likelihood:{}".format(time,LFun(probZ,probRZM,ratingM)))
    store = np.zeros((4,269))
    for i in range(4):
        for t in range(269):
            store[i,t] = post(i,t,probZ,probRZM,ratingM)
            
    for j in range(62):
        for i in range(4):
            newup = 0
            newdown = 0
            for t in range(269):
                newdown += store[i,t]
                if(ratingM[t,j] ==1):
                    newup += store[i,t]
                elif (ratingM[t,j]==-1):
                    newup += store[i,t]*probRZM[j,i]
            probRZM[j,i] = newup / newdown    
    for i in range(4):
        newvalue = 0
        for t in range(269):
            newvalue += store[i,t]
        newvalue = newvalue / 269
        probZ[i] = newvalue
    

Iterations:0,log-likelihood:-26.678832965400435
Iterations:1,log-likelihood:-16.094668997711192
Iterations:2,log-likelihood:-14.287794027341253
Iterations:4,log-likelihood:-13.265082934492524
Iterations:8,log-likelihood:-12.847308711972167
Iterations:16,log-likelihood:-12.705998052491518
Iterations:32,log-likelihood:-12.640737126831329
Iterations:64,log-likelihood:-12.616074566973708
Iterations:128,log-likelihood:-12.591194247298994


In [116]:
myGoer = []
for i in range(62):
    if(ratingM[68,i]==-1):
        res = 0
        for j in range(4):
            res += post(j,68,probZ,probRZM,ratingM) * probRZM[i,j]
        myGoer.append((i,res))

In [126]:
myGoer = sorted(myGoer,key=lambda x:x[1],reverse=True)
for i in myGoer:
    print("My preference for movie <{}> might be {}".format(movies[i[0]],i[1]))

My preference for movie <Inception> might be 0.9954437773878834
My preference for movie <The_Dark_Knight_Rises> might be 0.9570888790882955
My preference for movie <Wolf_of_Wall_Street> might be 0.9337307304884398
My preference for movie <The_Social_Network> might be 0.9196822776484885
My preference for movie <Now_You_See_Me> might be 0.8899980713995929
My preference for movie <Django_Unchained> might be 0.889346787636565
My preference for movie <Shutter_Island> might be 0.8883747739633127
My preference for movie <Les_Miserables> might be 0.8848266148314107
My preference for movie <The_Theory_of_Everything> might be 0.8779581039049331
My preference for movie <Toy_Story_3> might be 0.8707688538645239
My preference for movie <Star_Wars:_The_Force_Awakens> might be 0.8629306178013846
My preference for movie <Manchester_by_the_Sea> might be 0.8585411568742193
My preference for movie <Ex_Machina> might be 0.8526107427742802
My preference for movie <Black_Swan> might be 0.8495287683510985
My